# 09_02: Summarizing distributions

In [ ]:
# incomes = pd.concat([pd.read_csv('../gdata/income-1965-china.csv').assign(country='China', year=1965),
#           pd.read_csv('../gdata/income-1965-usa.csv').assign(country='USA', year=1965),
#           pd.read_csv('../gdata/income-2015-china.csv').assign(country='China', year=2015),
#           pd.read_csv('../gdata/income-2015-usa.csv').assign(country='USA', year=2015)]).sort_values(['country','year']).convert_dtypes(dtype_backend='pyarrow')

# incomes.to_csv('incomes.csv', index=False)

In [1]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

To learn from data, we first need to summarize it: that is, we need to quantify both the typical trends, and the variations around them---in short, the distribution of the data.

In this video we'll look at a few simple ways to summarize the _distribution_ of a numerical variable. 

We'll play with gapminder data that describe the distribution of incomes in China and in the US, in 1965 and 2015; these data are in 2011-equivalent dollars. These distributions are not very accurate, but they will be sufficient for our example.

In [76]:
incomes = pd.read_csv('incomes2.csv').set_index(['country', 'year'])
incomes

yearly
country year              
China   1965    374.584412
        1965    332.899298
        1965     40.405060
        1965    171.425536
        1965    136.738420
...                    ...
USA     2015  17830.112511
        2015  12448.307718
        2015  87588.746752
        2015   5597.465988
        2015  36709.679066

[4000 rows x 1 columns]

In [175]:
incomes.groupby(['country','year']).count()

yearly  log10_daily
country year                     
China   1965    1000         1000
        2015    1000         1000
USA     1965    1000         1000
        2015    1000         1000

For each country and year, we have 1,000 entries, corresponding to a sample of 1,000 representative people.

Hans Rosling argues that it is most insightful to look at the base-10 logarithm of the daily income. He points out that the actual difference that money makes in one's quality of life goes roughly logarithmically with daily income. For instance, if you have 16 dollars a day, you have to double it before things really change for you.

No problem, that's easy to compute.

In [78]:
incomes['log10_daily'] = np.log10(incomes.yearly / 365.25)

In [79]:
incomes

yearly  log10_daily
country year                           
China   1965    374.584412     0.010959
        1965    332.899298    -0.040277
        1965     40.405060    -0.956154
        1965    171.425536    -0.328515
        1965    136.738420    -0.426700
...                    ...          ...
USA     2015  17830.112511     1.688564
        2015  12448.307718     1.532520
        2015  87588.746752     2.379858
        2015   5597.465988     1.185401
        2015  36709.679066     2.002190

[4000 rows x 2 columns]

One way to describe the variation of a variable is by quantifying its range (more precisely, its **range of extremes**). However, focusing on the extremes is usually not very insightful; it is also imprecise given that our dataset is a limited sample of a population, rather than a complete census. Nevertheless, you get minimum and maxima in pandas with the min and max methods, applied to a DataFrame or to a Series.

In [81]:
incomes.loc['USA', 2015].min()

yearly         1782.589480
log10_daily       0.688461
dtype: float64

In [82]:
incomes.loc['USA', 2015].max()

yearly         256054.350614
log10_daily         2.845742
dtype: float64

Both minimum and maximum are **statistics**: descriptive numbers that we compute from the data, and that summarize them. Of course, another very important statistic is the **mean**, computed by summing up all the datapoints and dividing by the number of points. In symbols, we'd write a sum of the data points x_i divided by their number.

**mean**: $\bar{x} = (\sum_{i=1}^{N} x_i) / N$

In [95]:
incomes.loc['USA', 2015].mean()

yearly         22159.404538
log10_daily        1.666931
dtype: float64

We can group by the index to compute a statistic for both countries and years:

In [89]:
incomes.groupby(['country', 'year']).yearly.mean()

country  year
China    1965      241.117733
         2015     3651.785585
USA      1965    11529.607090
         2015    22159.404538
Name: yearly, dtype: float64

The **variance** is a measure of variation tied closely to the mathematical concept of normal distribution. To compute it, we obtain deviations from the mean, square them, and take the average. For a technical reason, we actually divide by the number of points minus one.

The square root of the variance is known as **standard deviation**, and taken together with the mean, it gives a pretty good idea of the center and variation of a distribution.

**Variance**: ${\rm var}(x) = \frac{1}{N - 1} \sum_{i=0}^N (x_i - \bar{x})^2$

**Standard deviation**: ${\rm std}(x) = \sqrt{{\rm var}(x)}$

In [93]:
incomes.groupby(['country', 'year']).var()

yearly  log10_daily
country year                           
China   1965  2.785142e+04     0.088698
        2015  7.657566e+06     0.091214
USA     1965  6.507744e+07     0.068749
        2015  3.836775e+08     0.098663

In [92]:
incomes.groupby(['country', 'year']).std()

yearly  log10_daily
country year                           
China   1965    166.887455     0.297822
        2015   2767.230725     0.302017
USA     1965   8067.058766     0.262200
        2015  19587.688162     0.314107

The **quantile** is a statistic that describes the value for which a certain percentage of the datapoints lies below. We compute it as follows. Taken together, the 25% and 75% quantiles specify a **coverage interval** that includes 50% of the datapoints. The 50% quantile is known as the median.

From this table, we see, say, that in 2015 approximately half of the Chinese population made less than 2869 dollars, and approximately 25% made less than $1839.

In [127]:
pd.DataFrame({'25%': incomes.groupby(['country', 'year']).yearly.quantile(0.25),
              '50%': incomes.groupby(['country', 'year']).yearly.median(),
              '75%': incomes.groupby(['country', 'year']).yearly.quantile(0.75)})

25%           50%           75%
country year                                          
China   1965    125.607498    203.478954    315.248787
        2015   1839.690284   2869.259934   4659.306189
USA     1965   6386.986126   9515.378686  14241.246328
        2015  10771.205045  16859.098812  26648.526889

The inverse of the quantile consists in finding the percentage of the population at which a given value (or **score**) lies. To find it, we need to go outside pandas, to scipy.stats. To compute a non-pandas statistic with groupby, we use `apply`.

In the China of 1965, no one in our sample made 5000 dollars a year. By contrast, it is a very low income in the States in 2015.

In [150]:
import scipy.stats

In [151]:
incomes.groupby(['country', 'year']).yearly.apply(lambda d: scipy.stats.percentileofscore(d, 5000))

country  year
China    1965    100.0
         2015     79.8
USA      1965     13.7
         2015      4.5
Name: yearly, dtype: float64

pandas offers a convenient method, `describe`, which returns several summary statistics at once. Between 1965 and 2015, China made great strides, but it had still some way to go. We see these clearly if we take USA/2015 values as a reference and use them to normalize the other entries.

In [162]:
incomes.groupby(['year', 'country']).yearly.describe() / incomes.loc['USA', 2015].yearly.describe()

count      mean       std       min       25%       50%  \
year country                                                            
1965 China      1.0  0.010881  0.008520  0.008593  0.011661  0.012069   
     USA        1.0  0.520303  0.411843  0.855450  0.592969  0.564406   
2015 China      1.0  0.164796  0.141274  0.104280  0.170797  0.170191   
     USA        1.0  1.000000  1.000000  1.000000  1.000000  1.000000   

                   75%       max  
year country                      
1965 China    0.011830  0.007736  
     USA      0.534410  0.350711  
2015 China    0.174843  0.076493  
     USA      1.000000  1.000000

Great. In the next video we'll move on to plotting these distributions.